# CNN

CNN implementation with Tensorflow

Sources:
https://www.tensorflow.org/tutorials/images/cnn

## Dependencies

In [ ]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
import matplotlib.pyplot as plt

## Load dataset
We will use the CIFAR10 dataset containing 60k color images in 10 classes, 6k per class.

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
# normalize pixel values
X_train = X_train.astype(float) / 255.
X_test = X_test.astype(float) / 255.

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

## Visualize dataset
Note that unlike mnist, each image is 32 x 32 and has 3 channels (rgb), hence 32 x 32 x 3

In [ ]:
print('Feature matrix:', X_train.shape)
print('Target matrix:', X_test.shape)
print('Feature matrix:', y_train.shape)
print('Target matrix:', y_test.shape)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[y_train[i][0]])
plt.show()

## Create our model
Our model will be processing images with RGB color channels hence each input image is of 32 x 32 x 3 dimension. Our convolutional base uses a common pattern: a stack of Conv2D and MaxPooling2D layers.

In [ ]:
model = Sequential([
    #
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)),
    Dropout(.2),
    MaxPooling2D(pool_size=(2, 2)),
    #
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    Dropout(.2),
    MaxPooling2D(pool_size=(2, 2)),
    #
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
])

Conv2D params:
- filters: dimensionality of the output space (how many output filters in the convolution)

View the architecture of the model so far:

In [ ]:
model.summary()

Note that all layers do not have padding applied hence the loss of 2 layers for a kernel of size 3 x 3.

The output of every Conv2D and MaxPooling2D is a 3D tensor of shape (height, width, channels). Number of output channels is controlled by the filter argument in Conv2D which specifies how many filters are applied to this layer. i.e. 32 filters = 32 channels.

As width and height shrinks it will be more computationally affordable to add more output channels in each Conv2D layer.

## Classification
To complete the model, we will feed the last output tensor (4,4,64) into one more more dense layers to perform classification. Flatten the 3D tensor into 1D before feeding it into the first dense layer.

In [ ]:
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

## Complete model
Compile with optimizer (Adam, SGD etc) and loss calculation (MSE etc)

In [ ]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

## Train

In [ ]:
history = model.fit(X_train, y_train, 
          epochs=10, 
          validation_split=0.2)

## Evaluate

accuracy: training data accuracy

val_accuracy: validation split accuracy

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

## Results

In [ ]:
results = model.evaluate(X_test, y_test, verbose=0)
print('test loss, test acc:', results)

## Results analysis
Default model from TensorFlow tutorial: 0.71